# Residential Rooftop PV Example

## Set up.

### Import packages.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../src"))

In [2]:
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import re                as re
import scipy.stats       as st
import seaborn           as sb
import tyche             as ty

from copy import deepcopy

## Scenario analyses.

### Load data.

#### The data are stored in a set of tab-separated value files in a folder.

In [3]:
designs = ty.Designs("data/pv_residential_validation")

#### Compile the production and metric functions for each technology in the dataset.

In [4]:
designs.compile()

ModuleNotFoundError: No module named 'technology.Types'

### Examine the data.

#### The `functions` table specifies where the Python code for each technology resides.

In [ ]:
designs.functions

Right now, only the style `numpy` is supported.

#### The `indices` table defines the subscripts for variables.

In [ ]:
designs.indices.sort_values(["Technology", "Type", "Offset"])

#### The `designs` table contains the cost, input, efficiency, and price data for a scenario.

In [ ]:
designs.designs.xs("2015 Actual", level="Scenario", drop_level=False)

#### The `parameters` table contains additional techno-economic parameters for each technology.

In [ ]:
designs.parameters.xs("2015 Actual", level="Scenario", drop_level=False).sort_values(["Technology", "Scenario", "Offset"])

#### The `results` table specifies the units of measure for results of computations.

In [ ]:
designs.results

### Evaluate the scenarios in the dataset.

In [ ]:
scenario_results = designs.evaluate_scenarios(sample_count=500)

In [ ]:
scenario_results

#### Plot the results.

In [ ]:
expert_results = scenario_results[["Value"]].xs(
    "LCOE", level="Index"
).rename(
    columns={"Value" : "LCOE [Δ$/kWh]"}
).unstack(
    ["Scenario"]
).xs("LCOE [Δ$/kWh]", axis=1, drop_level=True).reset_index(drop=True)
expert_results.plot.hist(bins=30)

### Make tornado plots for Expert A.

#### Remember base case LCOE.

In [ ]:
base_lcoe = 0.10613269974604357

#### Define the factors.

In [ ]:
tornado_factors = [
    "MCC", "MLT", "MEF", "MAP", "MOM",
    "MDR", "ICC", "ILT", "IRC", "IEF",
    "BCC", "BLR", "BPR", "BCA", "BOH",
]

#### Add the scenarios to the design.

In [ ]:
designs.parameters

In [ ]:
design_2015_actual    = designs.designs.xs   ("2015 Actual", level="Scenario")
parameter_2015_actual = designs.parameters.xs("2015 Actual", level="Scenario")
parameter_expert_a    = designs.parameters.xs("Expert A"   , level="Scenario")
for factor in tornado_factors:
    scenario_new = factor
    design_new = design_2015_actual.copy()
    design_new["Scenario"] = scenario_new
    designs.designs = designs.designs.append(design_new.reset_index().set_index(["Technology", "Scenario", "Variable", "Index"]))
    parameter_new = pd.concat([
        parameter_2015_actual[parameter_2015_actual["Notes"] != factor],
        parameter_expert_a   [parameter_expert_a   ["Notes"] == factor],
    ])
    parameter_new["Scenario"] = factor
    designs.parameters = designs.parameters.append(parameter_new.reset_index().set_index(["Technology", "Scenario", "Parameter"]))

#### Recompile the design.

In [ ]:
designs.compile()

#### Compute the results.

In [ ]:
scenario_results = designs.evaluate_scenarios(sample_count=500)
scenario_results.shape

#### Make the tornado plot.

In [ ]:
tornado_results =scenario_results[[
    "Value"
]].xs(
    "LCOE", level="Index"
).rename(
    columns={"Value" : "LCOE [$/kWh]"}
).reset_index(
    ["Technology", "Sample", "Variable"], drop=True
).drop(
    ["2015 Actual", "Expert A", "Expert B", "Expert C"]
).reset_index(
).sort_values(
    "LCOE [$/kWh]", ascending = False
)
tornado_results["LCOE Reduction [%]"] = 100 * tornado_results["LCOE [$/kWh]"] / 0.106125

In [ ]:
pl.figure(figsize=(8, 6), dpi = 300)

In [ ]:
sb.set(font_scale = 1)

In [ ]:
ax = sb.boxplot(
    data = tornado_results,
    y = "Scenario",
    x = "LCOE Reduction [%]"
)
for i,box in enumerate(ax.artists):
    box.set_edgecolor('black')
    box.set_facecolor('none')
    for j in range(6*i,6*(i+1)):
         ax.lines[j].set_color('black')

In [ ]:
z = list(ax.get_children())

In [ ]:
sb.barplot(
    data = tornado_results.groupby("Scenario").agg(np.mean).reset_index().sort_values(
    "LCOE [$/kWh]", ascending = False,
),
    y = "Scenario",
    x = "LCOE Reduction [%]",
    color = "lightblue",
    ax = ax,
)

In [ ]:
for zi in z:
    zi.set_zorder(1000)

In [ ]:
ax.set_xlabel("LCOE Reduction [%]",fontsize=15)
ax.set_ylabel("Scenario",fontsize=15)
ax.tick_params(labelsize=15)

In [ ]:
ax.figure

In [ ]:
ax.figure.savefig("pv_residential_validation.png", dpi = 300, bbox_inches = "tight")